In [17]:
import tensorflow as tf

In [18]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8877975983298907268
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7029653504
locality {
  bus_id: 1
  links {
  }
}
incarnation: 18220363421955477556
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1070 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 6.1"
xla_global_id: 416903419
]


In [19]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1
